# SVAMP Math Reasoning with GRPO Fine-tuning

This notebook fine-tunes **Gemma 3 1B-IT** on the SVAMP dataset using **Group Relative Policy Optimization (GRPO)**, a reinforcement learning technique that teaches the model to solve elementary arithmetic word problems with structured reasoning.

**Pipeline Overview**:
1. Load SVAMP dataset (1000 arithmetic word problems)
2. Initialize Gemma 3 1B with LoRA adapters
3. Define multi-component reward function
4. Train using GRPO on TPU v5e
5. Evaluate improvement on test set

**Key Innovation**: GRPO generates multiple responses per question and learns from relative quality comparisons, avoiding the need for human preference labels.

In [1]:
import os
os.environ["HF_HUB_DISABLE_XET"] = "1"

## 📦 Environment Setup

Installing dependencies for TPU-accelerated training with the Tunix framework (Google's RL toolkit for LLM fine-tuning).

**Key Libraries**:
- `google-tunix`: GRPO implementation and model utilities
- `flax==0.12.0`: Neural network framework compatible with JAX
- `datasets`: For loading SVAMP from Hugging Face
- `qwix`: LoRA (Low-Rank Adaptation) utilities

In [2]:
# Fix fsspec conflict first (this is blocking datasets)
!pip install -q "fsspec==2023.10.0"

# Install tunix and let it handle dependencies
!pip install -q --force-reinstall "google-tunix[prod]==0.1.3"

# Fix remaining conflicts
!pip install -q --upgrade "numpy>=2.0,<3.0"
!pip install -q --force-reinstall --no-deps pyarrow==16.1.0

# Install other required packages
!pip install -q kagglehub 
!pip install -q ipywidgets 
!pip install -q tensorflow 
!pip install -q tensorflow_datasets
!pip install -q tensorboardX
!pip install -q wandb

print("Installation complete!")

# Verify tunix is installed
import sys
import importlib.util

tunix_spec = importlib.util.find_spec("tunix")
if tunix_spec is None:
    print("ERROR: tunix is not installed!")
    print("Attempting to reinstall...")
    import subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "--force-reinstall", "google-tunix[prod]==0.1.3"], check=True)
else:
    print("✓ tunix is installed at:", tunix_spec.origin)

# Now import everything
print("\nImporting libraries...")

import functools
import gc
import os
from pprint import pprint
import re
import csv
import shutil

# Import JAX and related libraries
import jax
import jax.numpy as jnp
print("✓ JAX imported")

# Import Flax
from flax import nnx
print("✓ Flax imported")

# Import other core libraries
import grain
import humanize
import kagglehub
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm
print("✓ Core libraries imported")

# Import tunix modules - doing this step by step to see where it fails
try:
    import tunix
    print("✓ tunix base module imported")
    
    from tunix.generate import sampler as sampler_lib
    print("✓ tunix.generate.sampler imported")
    
    from tunix.generate import tokenizer_adapter as tokenizer_lib
    print("✓ tunix.generate.tokenizer_adapter imported")
    
    from tunix.models.gemma3 import params
    print("✓ tunix.models.gemma3.params imported")
    
    from tunix.models.gemma3 import model
    print("✓ tunix.models.gemma3.model imported")
    
    from tunix.rl import rl_cluster as rl_cluster_lib
    print("✓ tunix.rl.rl_cluster imported")
    
    from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
    print("✓ tunix.rl.grpo.grpo_learner imported")
    
    from tunix.rl.rollout import base_rollout
    print("✓ tunix.rl.rollout imported")
    
    from tunix.sft import metrics_logger
    print("✓ tunix.sft.metrics_logger imported")
    
except ImportError as e:
    print(f"ERROR importing tunix module: {e}")
    import traceback
    traceback.print_exc()

# Import datasets last
from datasets import load_dataset
print("✓ datasets imported")

print("\n" + "="*50)
print("ALL IMPORTS SUCCESSFUL!")
print("="*50)


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.63.1 requires numpy<2.4,>=1.22, but you have numpy 2.4.1 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To

/usr/local/lib/python3.12/site-packages/jax/_src/cloud_tpu_init.py:93: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


✓ JAX imported
✓ Flax imported
✓ Core libraries imported
✓ tunix base module imported
✓ tunix.generate.sampler imported
✓ tunix.generate.tokenizer_adapter imported
✓ tunix.models.gemma3.params imported
✓ tunix.models.gemma3.model imported
✓ tunix.rl.rl_cluster imported
✓ tunix.rl.grpo.grpo_learner imported
✓ tunix.rl.rollout imported
✓ tunix.sft.metrics_logger imported
✓ datasets imported

ALL IMPORTS SUCCESSFUL!


In [ ]:
'''# Install core libraries
!pip install -q kagglehub 
!pip install -q ipywidgets 
!pip install -q tensorflow 
!pip install -q tensorflow_datasets
!pip install -q tensorboardX
!pip install -q grain

# Install the Google Tunix framework (the "teacher's toolkit")
!pip install "google-tunix[prod]==0.1.3"

# Reinstall Flax to ensure compatibility
!pip uninstall -q -y flax
!pip install -q flax==0.12.0

# Fix huggingface-hub version conflict
!pip install -q --force-reinstall "huggingface-hub>=0.34.0,<1.0"

# Fix transformers to a compatible version
!pip install -q --force-reinstall transformers

# Fix datasets and pyarrow compatibility
!pip install -q --force-reinstall pyarrow==15.0.0
!pip install -q --force-reinstall datasets==2.19.0

# Install wandb for experiment tracking
!pip install -q wandb'''


In [3]:
# Set up Weights & Biases (for tracking training)
import wandb, os
from kaggle_secrets import UserSecretsClient
os.environ['WANDB_API_KEY'] = UserSecretsClient().get_secret("WANDB_API_KEY")
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")

## 🔧 Import Core Libraries

Setting up JAX for TPU computation, Flax for neural networks, and Tunix for GRPO training infrastructure.

In [ ]:
'''# Import all necessary libraries
import functools
import gc
import os
from pprint import pprint
import re
import csv
import shutil
from flax import nnx
import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm
from tunix.generate import sampler as sampler_lib
from tunix.generate import tokenizer_adapter as tokenizer_lib
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl.rollout import base_rollout
from tunix.sft import metrics_logger
from datasets import load_dataset

print("All imports successful!")
'''

In [ ]:
'''# First, let's install everything tunix needs, allowing it to resolve dependencies
!pip install -q "google-tunix[prod]==0.1.3"

# Now fix the specific conflicts that arose
!pip install -q --upgrade "numpy>=2.0,<3.0"
!pip install -q --force-reinstall --no-deps pyarrow==16.1.0
!pip install -q --force-reinstall flax==0.12.0

# Reinstall other required packages
!pip install -q kagglehub 
!pip install -q ipywidgets 
!pip install -q tensorflow 
!pip install -q tensorflow_datasets
!pip install -q tensorboardX
!pip install -q grain
!pip install -q wandb

print("Installation complete! Now importing...")

# Import all necessary libraries
import functools
import gc
import os
from pprint import pprint
import re
import csv
import shutil

# Import JAX and related libraries
import jax
import jax.numpy as jnp

# Import other core libraries
import grain
import humanize
import kagglehub
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm

# Import Flax
from flax import nnx

# Import tunix modules
from tunix.generate import sampler as sampler_lib
from tunix.generate import tokenizer_adapter as tokenizer_lib
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl.rollout import base_rollout
from tunix.sft import metrics_logger

# Import datasets last
from datasets import load_dataset

print("✓ All imports successful!")'''

## ⚙️ Hyperparameter Configuration

Comprehensive training configuration organized by category.

### 📊 Data Parameters
- **TRAIN_FRACTION**: Using 100% of training data
- **MESH**: Distributed training layout (1×4 = FSDP × Tensor Parallel)

### 🎯 LoRA Parameters
- **RANK=64, ALPHA=64**: Controls adapter capacity
- Enables training <1% of parameters while maintaining performance

### 🎲 GRPO Algorithm
- **NUM_GENERATIONS=4**: Generate 4 responses per question for comparison
- **BETA=0.04**: KL divergence penalty (prevents drift from base model)
- **EPSILON=0.2**: PPO clipping parameter for stable updates
- **TOTAL_GENERATION_STEPS=384**: Allows detailed reasoning chains

### 📈 Optimization
- **LEARNING_RATE=3e-6**: Conservative for RL stability
- **WARMUP_STEPS=170**: Gradual learning rate ramp-up
- **MAX_GRAD_NORM=0.1**: Gradient clipping for training stability
- **NUM_BATCHES=1000**: Total training iterations

### 💾 Infrastructure
- **Checkpointing**: Save every 500 steps, keep 4 most recent
- **Evaluation**: Test model every 10 steps

In [4]:
# ====== Data ======
TRAIN_DATA_DIR = "./data/train"
TEST_DATA_DIR = "./data/test"
TRAIN_FRACTION = 1.0

# ====== LoRA ======
RANK = 64
ALPHA = 64.0

# ====== Sharding ======
MESH = [(1, 4), ("fsdp", "tp")]

# ====== GRPO ======
MAX_PROMPT_LENGTH = 256
TOTAL_GENERATION_STEPS = 384  # Increased for longer math solutions
TEMPERATURE = 0.9
TOP_P = 1.0
TOP_K = 50
NUM_GENERATIONS = 4

# ====== Training ======
NUM_ITERATIONS = 1
BETA = 0.04
EPSILON = 0.2

TRAIN_MICRO_BATCH_SIZE = 2
NUM_BATCHES = 1000  
NUM_TEST_BATCHES = 100
EVAL_EVERY_N_STEPS = 10
NUM_EPOCHS = 2

MAX_STEPS = int(NUM_BATCHES * NUM_ITERATIONS * TRAIN_FRACTION * NUM_EPOCHS)

# ====== Optimizer ======
LEARNING_RATE = 3e-6
B1 = 0.9
B2 = 0.99
WEIGHT_DECAY = 0.1
WARMUP_STEPS = 0.1 * MAX_STEPS
MAX_GRAD_NORM = 0.1

# ====== Checkpointing ======
INTERMEDIATE_CKPT_DIR = "/tmp/content/intermediate_ckpt/"
CKPT_DIR = "/tmp/content/ckpts/"
SAVE_INTERVAL_STEPS = 500
MAX_TO_KEEP = 4

# ====== Inference ======
GENERATION_CONFIGS = {
    "greedy": {"temperature": 1e-4, "top_k": 1, "top_p": 1.0},
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}

print(f"Total training steps: {MAX_STEPS}")

Total training steps: 2000


In [5]:
def show_hbm_usage():
    """Displays memory usage per device."""
    fmt_size = functools.partial(humanize.naturalsize, binary=True)
    
    for d in jax.local_devices():
        stats = d.memory_stats()
        used = stats["bytes_in_use"]
        limit = stats["bytes_limit"]
        print(f"Using {fmt_size(used)} / {fmt_size(limit)} ({used/limit:%}) on {d}")

## 🎯 Prompt Engineering for SVAMP

Designing the system prompt that teaches the model our desired output format.

**Format Requirements**:
1. `<reasoning>` tags: Step-by-step mathematical thinking
2. `<answer>` tags: Final numerical answer only

This structured format enables precise reward calculation and ensures interpretable solutions.

In [6]:
reasoning_start = "<reasoning>"
reasoning_end = "</reasoning>"
solution_start = "<answer>"
solution_end = "</answer>"

# UPDATED: System prompt for SVAMP math word problems
SYSTEM_PROMPT = f"""You are a mathematical reasoning expert specializing in solving arithmetic word problems.
Your goal is to solve problems by breaking them down into logical steps.

You must strictly follow this format:
1. Start with {reasoning_start}.
2. Write out your step-by-step solution with clear mathematical reasoning.
3. Show all calculations and explain each step.
4. End reasoning with {reasoning_end}.
5. Provide the final numerical answer between {solution_start} and {solution_end}.

Example:
User: Rachel had to complete 5 pages of math homework. If she had to complete 4 more pages of reading homework than math homework, how many pages did she have to complete in all?

Model:
{reasoning_start}
Step 1: Rachel has 5 pages of math homework.
Step 2: She has 4 more pages of reading than math, so reading = 5 + 4 = 9 pages.
Step 3: Total pages = math + reading = 5 + 9 = 14 pages.
{reasoning_end}
{solution_start}14{solution_end}

Now solve the problem below using this exact format."""

TEMPLATE = """<start_of_turn>user
{system_prompt}

{question}<end_of_turn>
<start_of_turn>model"""

print("✅ System prompt configured for SVAMP dataset")

✅ System prompt configured for SVAMP dataset


## 📚 Dataset Loading and Preprocessing

SVAMP (Simple Variations on Arithmetic Math word Problems) contains 1000 elementary math word problems requiring 1-2 arithmetic operations.

**Preprocessing Steps**:
1. Combine `Body` and `Question` fields
2. Extract numerical `Answer`
3. Format with system prompt template
4. **Curriculum Learning**: Sort by equation complexity (easy→hard) to improve training efficiency

In [7]:
def extract_svamp_answer(text: str) -> str | None:
    """Extract the numerical answer from SVAMP response."""
    # Look for answer between tags
    match = re.search(rf'{solution_start}\s*([\d.]+)\s*{solution_end}', text)
    if match:
        return match.group(1).strip()
    
    # Fallback: look for any number after "answer" keyword
    match = re.search(r'answer.*?([\d.]+)', text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    
    return None

def get_svamp_dataset(split="train"):
    """Load and preprocess SVAMP dataset."""
    print(f"Loading SVAMP dataset split: {split}")
    
    # Load from Hugging Face
    dataset = load_dataset("ChilleD/SVAMP", split="train")
    
    # Split into train/test (80-20)
    if split == "train":
        dataset = dataset.select(range(int(len(dataset) * 0.8)))
    else:  # test
        dataset = dataset.select(range(int(len(dataset) * 0.8), len(dataset)))
    
    def preprocess(example):
        # Combine body and question
        question = f"{example['Body']} {example['Question']}"
        
        # Get answer (convert to string)
        answer = str(example['Answer'])
        
        return {
            "prompts": TEMPLATE.format(
                system_prompt=SYSTEM_PROMPT,
                question=question
            ),
            "question": question,
            "answer": answer,
            "equation": example.get('Equation', ''),
            "type": example.get('Type', '')
        }
    
    # Convert to grain dataset
    data = [preprocess(item) for item in dataset]
    
    # Sort by equation complexity for curriculum learning (easy to hard)
    data.sort(key=lambda x: len(str(x.get("equation", ""))))
    print(f"✅ Data sorted by complexity for Curriculum Learning")
    
    grain_dataset = grain.MapDataset.source(data)
    
    return grain_dataset

def get_dataset(data_dir, split="train", source="huggingface"):
    """Wrapper function to maintain compatibility."""
    return get_svamp_dataset(split=split)

### Loading and Splitting Data

Creating 80/20 train/test split with batching for efficient TPU processing.

In [8]:
# Load SVAMP datasets
print("Loading SVAMP training data...")
dataset = get_svamp_dataset("train").batch(TRAIN_MICRO_BATCH_SIZE)[:NUM_BATCHES]

if TRAIN_FRACTION == 1.0:
    train_dataset = dataset.repeat(NUM_EPOCHS)
    val_dataset = None
else:
    train_dataset = dataset[:int(len(dataset) * TRAIN_FRACTION)]
    train_dataset = train_dataset.repeat(NUM_EPOCHS)
    val_dataset = dataset[int(len(dataset) * TRAIN_FRACTION):].repeat(NUM_EPOCHS)

print("Loading SVAMP test data...")
test_dataset = get_svamp_dataset("test").batch(TRAIN_MICRO_BATCH_SIZE)[:NUM_TEST_BATCHES]

dataset_lengths = (
    len(train_dataset),
    len(val_dataset) if val_dataset is not None else 0,
    len(test_dataset),
)
print(f"✅ Dataset contains {dataset_lengths} batches (train, val, test)")

# Show a sample
print("\n📋 Sample from training data:")
for ele in train_dataset[:1]:
    pprint(ele)

Loading SVAMP training data...
Loading SVAMP dataset split: train


README.md:   0%|          | 0.00/675 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/111k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/700 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

✅ Data sorted by complexity for Curriculum Learning
Loading SVAMP test data...
Loading SVAMP dataset split: test
✅ Data sorted by complexity for Curriculum Learning
✅ Dataset contains (560, 0, 70) batches (train, val, test)

📋 Sample from training data:
{'answer': array(['8', '3'], dtype='<U1'),
 'equation': array(['8.0', '( 6.0 / 2.0 )'], dtype='<U13'),
 'prompts': array(['<start_of_turn>user\nYou are a mathematical reasoning expert specializing in solving arithmetic word problems.\nYour goal is to solve problems by breaking them down into logical steps.\n\nYou must strictly follow this format:\n1. Start with <reasoning>.\n2. Write out your step-by-step solution with clear mathematical reasoning.\n3. Show all calculations and explain each step.\n4. End reasoning with </reasoning>.\n5. Provide the final numerical answer between <answer> and </answer>.\n\nExample:\nUser: Rachel had to complete 5 pages of math homework. If she had to complete 4 more pages of reading homework than math ho

## 🔑 Kaggle Authentication

Configuring Kaggle credentials for model downloads and artifact management.

In [9]:
import os
import json

# 1. DEFINE YOUR CREDENTIALS
# (Make sure there are no spaces around the strings!)
kaggle_username = "rachitha02"
kaggle_key = "Rachitha CB"

# 2. SETUP THE DIRECTORY
# This creates the hidden .kaggle folder if it doesn't exist
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# 3. FORCE WRITE THE FILE
# This overwrites any broken file that might be causing the 401
json_path = os.path.join(kaggle_dir, "kaggle.json")
with open(json_path, "w") as f:
    json.dump({"username": kaggle_username, "key": kaggle_key}, f)

# 4. SET PERMISSIONS (Linux/Mac requirement for safety)
os.chmod(json_path, 0o600)

print("✅ Credentials successfully written to", json_path)

✅ Credentials successfully written to /root/.kaggle/kaggle.json


## 🤖 Base Model Initialization

Loading **Gemma 3 1B-IT** (instruction-tuned variant) as our starting point.

**Process**:
1. Load pre-trained model from Kaggle
2. Save to intermediate checkpoint
3. Free memory for training setup

The instruction-tuned variant already understands chat formatting and following instructions, making it ideal for our structured reasoning task.

In [10]:
# Clean up checkpoint directories
!rm /tmp/content/intermediate_ckpt/* -rf
!rm /tmp/content/ckpts/* -rf

# Load Gemma 3 base model
import gc
from orbax import checkpoint as ocp
from flax import nnx
from tunix.models.gemma3 import params
from tunix.models.gemma3 import model

print("🔄 Loading Gemma 3 1B-IT base model...")

model_family = "gemma3"
if model_family == "gemma3":
    MODEL_CP_PATH = params.GEMMA3_1B_IT
    config = model.ModelConfig.gemma3_1b()
    gemma = params.create_model_from_checkpoint(MODEL_CP_PATH, config)
    tokenizer = params.create_tokenizer()
    
    # Save intermediate checkpoint
    checkpointer = ocp.StandardCheckpointer()
    _, state = nnx.split(gemma)
    checkpointer.save(os.path.join(INTERMEDIATE_CKPT_DIR, "state"), state)
    checkpointer.wait_until_finished()
    
    print("✅ Base model loaded and saved to intermediate checkpoint")
    
    # Delete intermediate model to save memory
    del params
    del gemma
    del state
    gc.collect()
    
    print("✅ Memory cleaned up")

🔄 Loading Gemma 3 1B-IT base model...


E0000 00:00:1768083150.885141    1294 common_lib.cc:650] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:238
E0110 22:12:56.547338    2512 google_auth_provider.cc:188] Could not find the credentials file in the standard gcloud location [/root/.config/gcloud/application_default_credentials.json]. You may specify a credentials file using $GOOGLE_APPLICATION_CREDENTIALS, or to use Google application default credentials, run: gcloud auth application-default login


✅ Base model loaded and saved to intermediate checkpoint
✅ Memory cleaned up


In [11]:
# Verify checkpoint was saved
import os
print("📁 Checkpoint contents:", os.listdir(INTERMEDIATE_CKPT_DIR))
# Expected output: ['state', ...] or similar files

📁 Checkpoint contents: ['state']


## 🔄 Reference Model Setup

Creating a **frozen copy** of the base model for KL divergence calculation.

**Purpose**: 
- Measures how much the policy deviates from original behavior
- Prevents catastrophic forgetting of language abilities
- Core component of GRPO that balances task learning with coherent generation

**LoRA Application**:
Adds trainable low-rank matrices to attention and feedforward layers while keeping base weights frozen.

In [12]:
from tunix.models.gemma3 import params

def get_gemma_ref_model(ckpt_path):
    """Load reference model with sharding across TPU chips."""
    mesh = jax.make_mesh(*MESH)
    model_config = model.ModelConfig.gemma3_1b()
    
    # Create abstract model structure
    abs_gemma: nnx.Module = nnx.eval_shape(
        lambda: params.create_model_from_checkpoint(MODEL_CP_PATH, config)
    )
    
    abs_state = nnx.state(abs_gemma)
    abs_state = jax.tree.map(
        lambda a, s: jax.ShapeDtypeStruct(a.shape, jnp.bfloat16, sharding=s),
        abs_state,
        nnx.get_named_sharding(abs_state, mesh),
    )
    
    # Restore from checkpoint
    checkpointer = ocp.StandardCheckpointer()
    restored_params = checkpointer.restore(ckpt_path, target=abs_state)
    
    graph_def, _ = nnx.split(abs_gemma)
    gemma = nnx.merge(graph_def, restored_params)
    return gemma, mesh, model_config

def get_lora_model(base_model, mesh):
    """Apply LoRA adapters to base model."""
    lora_provider = qwix.LoraProvider(
        module_path=(
            ".*q_einsum|.*kv_einsum|.*gate_proj|.*down_proj|.*up_proj|"
            ".*attn_vec_einsum"
        ),
        rank=RANK,
        alpha=ALPHA,
    )
    
    model_input = base_model.get_model_input()
    lora_model = qwix.apply_lora_to_model(
        base_model, lora_provider, **model_input
    )
    
    # Shard the model across TPU
    with mesh:
        state = nnx.state(lora_model)
        pspecs = nnx.get_partition_spec(state)
        sharded_state = jax.lax.with_sharding_constraint(state, pspecs)
        nnx.update(lora_model, sharded_state)
    
    return lora_model

In [13]:
# Load reference model (frozen, for KL divergence calculation)
if model_family == "gemma3":
    ref_model, mesh, model_config = get_gemma_ref_model(
        ckpt_path=os.path.join(INTERMEDIATE_CKPT_DIR, "state")
    )
    print("✅ Reference model loaded")

/tmp/ipykernel_1294/359622891.py:5: DeprecationWarning: The default axis_types will change in JAX v0.9.0 to jax.sharding.AxisType.Explicit. To maintain the old behavior, pass `axis_types=(jax.sharding.AxisType.Auto,) * len(axis_names)`. To opt-into the new behavior, pass `axis_types=(jax.sharding.AxisType.Explicit,) * len(axis_names)
  mesh = jax.make_mesh(*MESH)
ERROR:asyncio:Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7dae85323d00> is already entered
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-3740' coro=<_async_in_context.<locals>.run_in_context() done, defined at /usr/local/lib/python3.12/site-packages/ipykernel/utils.py:57> wait_for=<Task pending name='Task-3741' coro=<Kernel.shell_main() running 

✅ Reference model loaded


### Flax NNX Compatibility Patch

Applying a compatibility fix for Flax API changes between versions.

In [14]:
# Apply compatibility patch for Flax NNX
from flax import nnx

# Save original function
if not hasattr(nnx.Variable, "_original_set_metadata"):
    nnx.Variable._original_set_metadata = nnx.Variable.set_metadata

# Define patched function
def patched_set_metadata(self, *args, **kwargs):
    """Fix for Flax NNX API changes."""
    if len(args) == 2 and isinstance(args[0], str):
        key = args[0]
        value = args[1]
        kwargs[key] = value
        args = ()
    return nnx.Variable._original_set_metadata(self, *args, **kwargs)

# Apply patch
nnx.Variable.set_metadata = patched_set_metadata

print("✅ Flax compatibility patch applied successfully!")

✅ Flax compatibility patch applied successfully!


### Applying LoRA to Create Policy Model

Wrapping the reference model with trainable LoRA adapters and sharding across TPU cores for distributed training.

In [15]:
# Create policy model with LoRA adapters
lora_policy = get_lora_model(ref_model, mesh=mesh)
print("✅ Policy model with LoRA created")
print("\n🧠 Model structure:")
# nnx.display(lora_policy)  # Uncomment to see model architecture

# Show memory usage
show_hbm_usage()

/usr/local/lib/python3.12/site-packages/qwix/_src/providers/lora.py:43: UserWarning: rngs must be provided for NNX models to initialize LoRA weights. Please specify rngs=nnx.Rngs(...) in apply_lora_to_model.
  warnings.warn(


✅ Policy model with LoRA created

🧠 Model structure:
Using 1.0 GiB / 15.7 GiB (6.501034%) on TPU_0(process=0,(0,0,0,0))
Using 1.0 GiB / 15.7 GiB (6.476935%) on TPU_1(process=0,(1,0,0,0))
Using 1.0 GiB / 15.7 GiB (6.476935%) on TPU_2(process=0,(0,1,0,0))
Using 1.0 GiB / 15.7 GiB (6.476935%) on TPU_3(process=0,(1,1,0,0))
Using 26.5 KiB / 15.7 GiB (0.000160%) on TPU_4(process=0,(0,2,0,0))
Using 26.5 KiB / 15.7 GiB (0.000160%) on TPU_5(process=0,(1,2,0,0))
Using 26.5 KiB / 15.7 GiB (0.000160%) on TPU_6(process=0,(0,3,0,0))
Using 26.5 KiB / 15.7 GiB (0.000160%) on TPU_7(process=0,(1,3,0,0))


## 🏆 Multi-Component Reward Function Design

GRPO requires reward functions to score model outputs. We use **7 complementary reward components** that together encourage correct, well-formatted, and well-reasoned solutions.

### Reward Breakdown:

**Format Rewards** (up to 3.5 points):
- `match_format_exactly`: +3.0 for perfect tag structure
- `match_format_approximately`: +2.0 for partial tag compliance

**Correctness Rewards** (up to 5.0 points):
- `check_answer_svamp`: +3.0 for exact numerical match
- `check_number_svamp`: +2.0 for extractable correct number

**Quality Rewards** (up to 2.5 points):
- `soft_reasoning_steps`: +1.0 for logical connectors ("first", "then", "therefore")
- `meaningful_reasoning_length`: +1.0 for appropriate length (15-400 words)
- `reward_algebraic_notation`: +1.5 for mathematical notation (equations, variables)

**Total Maximum**: ~11 points per response

This multi-faceted approach ensures the model learns both task-specific skills (correctness) and generalizable reasoning patterns (structure, logic).

In [16]:
# Regex patterns for matching format and answers
match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{reasoning_start}.+?{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL,
)

# For extracting numerical answers
match_number = re.compile(
    rf"{solution_start}.*?([\d.]+)", flags=re.MULTILINE | re.DOTALL
)

def match_format_exactly(prompts, completions, **kwargs):
    """Reward for exact format compliance."""
    return [
        0 if match_format.search(response) is None else 3.0
        for response in completions
    ]

def match_format_approximately(prompts, completions, **kwargs):
    """Reward for approximate format compliance."""
    scores = []
    for completion in completions:
        score = 0
        response = completion
        # Reward seeing each tag exactly once
        score += 0.5 if response.count(reasoning_start) == 1 else -0.5
        score += 0.5 if response.count(reasoning_end) == 1 else -0.5
        score += 0.5 if response.count(solution_start) == 1 else -0.5
        score += 0.5 if response.count(solution_end) == 1 else -0.5
        scores.append(score)
    return scores

def check_answer_svamp(prompts, completions, answer, **kwargs):
    """Reward for correct numerical answer."""
    responses = completions
    
    extracted_responses = [
        guess.group(1).strip() 
        if (guess := match_format.search(r)) is not None 
        else None
        for r in responses
    ]
    
    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        
        try:
            # Convert to float for comparison
            guess_num = float(guess)
            true_num = float(true_answer)
            
            # Exact match gets full points
            if abs(guess_num - true_num) < 0.01:  # Allow small floating point error
                score += 3.0
            # Close match gets partial credit
            elif abs(guess_num - true_num) < 1.0:
                score += 1.5
            else:
                score -= 1.0  # Penalize wrong answers
        except:
            score = 0
        
        scores.append(score)
    return scores

def check_number_svamp(prompts, completions, answer, **kwargs):
    """Extract and check numerical answer."""
    question = kwargs.get("question", [])
    responses = completions
    
    extracted_responses = [
        guess.group(1).strip() 
        if (guess := match_number.search(r)) is not None 
        else None
        for r in responses
    ]
    
    scores = []
    print("START ============================")
    if len(question) > 0:
        print(f"Question: {question[0][:100]}...")
    if len(answer) > 0:
        print(f"Correct Answer: {answer[0]}")
    if len(responses) > 0:
        print(f"Response: {responses[0][:200]}...")
    if len(extracted_responses) > 0:
        print(f"Extracted: {extracted_responses[0]}")
    print("END ==============================")
    
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        
        try:
            # Compare numbers
            if abs(float(guess) - float(true_answer)) < 0.01:
                scores.append(2.0)
            else:
                scores.append(0.0)
        except:
            scores.append(0.0)
    
    return scores

def soft_reasoning_steps(prompts, completions, **kwargs):
    """Reward for using logical connector words."""
    rewards = []
    logical_keywords = [
        "step", "first", "next", "then", "therefore", 
        "because", "since", "so", "implies", "consequently",
        "thus", "hence", "given", "solving"
    ]
    
    for response in completions:
        match = re.search(r"<reasoning>(.+?)</reasoning>", response, flags=re.DOTALL)
        if match:
            reasoning_text = match.group(1).lower()
            found_keywords = sum(1 for word in logical_keywords if word in reasoning_text)
            # 0.1 reward per keyword, capped at 1.0
            rewards.append(min(1.0, found_keywords * 0.1))
        else:
            rewards.append(0.0)
    return rewards

def meaningful_reasoning_length(prompts, completions, **kwargs):
    """Reward for appropriate reasoning length."""
    rewards = []
    for response in completions:
        match = re.search(r"<reasoning>(.+?)</reasoning>", response, flags=re.DOTALL)
        if match:
            word_count = len(match.group(1).split())
            
            if word_count < 15:  # Too short
                rewards.append(0.0)
            elif word_count > 400:  # Too long
                rewards.append(0.5)
            else:  # Good length
                rewards.append(1.0)
        else:
            rewards.append(0.0)
    return rewards

def reward_algebraic_notation(prompts, completions, **kwargs):
    """Reward for using algebraic notation and equations."""
    rewards = []
    algebra_patterns = [
        r'[a-z]\s*[+\-*/=]\s*\d',  # x + 5
        r'\d\s*[+\-*/=]\s*[a-z]',  # 5 + x
        r'[a-z]\s*=\s*',           # x =
        r'\([^)]*[a-z][^)]*\)',    # (x + 5)
    ]
    
    for response in completions:
        match = re.search(r"<reasoning>(.+?)</reasoning>", response, flags=re.DOTALL)
        if match:
            reasoning_text = match.group(1)
            found_patterns = sum(
                1 for pattern in algebra_patterns 
                if re.search(pattern, reasoning_text)
            )
            rewards.append(min(1.5, found_patterns * 0.3))
        else:
            rewards.append(0.0)
    return rewards

## 🎲 Generation Utilities

Helper functions for model inference and evaluation.

In [17]:
def generate(
    question, sampler, temperature=0.7, top_k=50, top_p=0.95, seed=None, options=None
):
    """Generate text given a prompt."""
    
    if isinstance(question, str):
        # Single question - SVAMP doesn't have options
        input_batch = [
            TEMPLATE.format(
                system_prompt=SYSTEM_PROMPT,
                question=question
            ),
        ]
    else:
        # Batch of questions
        input_batch = [
            TEMPLATE.format(
                system_prompt=SYSTEM_PROMPT,
                question=q
            )
            for q in question
        ]
    
    out_data = sampler(
        input_strings=input_batch,
        max_generation_steps=TOTAL_GENERATION_STEPS,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        echo=False,
        seed=seed if seed is not None else None,
        eos_tokens=[1, 106],
    )
    
    output = out_data.text
    if isinstance(question, str):
        return output[0]
    return output

In [18]:
def evaluate(
    dataset,
    sampler,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_passes=1,
    corr_lst=False,
    make_lst=False,
):
    """Compute accuracy and format compliance."""
    
    response_lst = []
    corr = 0
    partially_corr = 0
    corr_format = 0
    total = 0
    
    for batch in tqdm(dataset):
        answers = batch["answer"]
        questions = batch["question"]
        
        multiple_call_responses = [[] for _ in range(len(questions))]
        for p in range(num_passes):
            responses = generate(
                questions, sampler, temperature, top_k, top_p, seed=p, options=None
            )
            for idx, response in enumerate(responses):
                multiple_call_responses[idx].append(response)
        
        for question, multiple_call_response, answer in zip(
            questions, multiple_call_responses, answers
        ):
            corr_ctr_per_question = 0
            partially_corr_per_question = 0
            corr_format_per_question = 0
            
            for response in multiple_call_response:
                # Extract numerical answer
                extracted_response = (
                    guess.group(1).strip()
                    if (guess := match_number.search(response)) is not None
                    else None
                )
                
                # Check correctness
                if extracted_response:
                    try:
                        if abs(float(extracted_response) - float(answer)) < 0.01:
                            corr_ctr_per_question += 1
                            partially_corr_per_question += 1
                    except:
                        pass
                
                # Check format
                if match_format.search(response) is not None:
                    corr_format_per_question += 1
                
                if (
                    corr_ctr_per_question > 0
                    and partially_corr_per_question > 0
                    and corr_format_per_question > 0
                ):
                    break
            
            if corr_ctr_per_question > 0:
                corr += 1
                if corr_lst and make_lst:
                    response_lst.append((question, answer, multiple_call_response))
            else:
                if not corr_lst and make_lst:
                    response_lst.append((question, answer, multiple_call_response))
            
            if partially_corr_per_question > 0:
                partially_corr += 1
            if corr_format_per_question > 0:
                corr_format += 1
            
            total += 1
            if total % 10 == 0:
                print(
                    f"===> {corr=}, {total=}, Acc={corr / total * 100:.2f}%, "
                    f"Partial={partially_corr / total * 100:.2f}%, Format={corr_format / total * 100:.2f}%"
                )
    
    to_return = (
        corr,
        total,
        corr / total * 100,
        partially_corr / total * 100,
        corr_format / total * 100,
    )
    if make_lst:
        return to_return, response_lst
    return to_return

### Creating Sampler for Inference

Initializing the generation engine with KV-cache for efficient sequential decoding.

In [19]:
# Create sampler for generation
sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

print("✅ Sampler created successfully")

✅ Sampler created successfully


## 📊 Baseline Evaluation (Pre-Training)

Testing the **untrained** Gemma 3 1B-IT model to establish our improvement benchmark.

**Metrics**:
- **Accuracy**: Exact numerical answer correctness
- **Partial**: Any correct numerical value extracted
- **Format**: Proper `<reasoning>` and `<answer>` tag structure

Expected baseline: ~50-60% accuracy (Gemma 3 already has some math ability from pre-training)

In [20]:
# PRE-TRAINING EVALUATION
print("\n" + "="*60)
print("📊 EVALUATING MODEL BEFORE TRAINING")
print("="*60)
print("⏳ This will take 3-5 minutes. Patience is a virtue!")
print()

(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset,
    sampler,
    **GENERATION_CONFIGS["greedy"],
)

print("\n" + "="*60)
print("📈 PRE-TRAINING RESULTS:")
print(f"   Correct answers: {corr}/{total}")
print(f"   Accuracy: {accuracy:.2f}%")
print(f"   Partial accuracy: {partial_accuracy:.2f}%")
print(f"   Format compliance: {format_accuracy:.2f}%")
print("="*60)


📊 EVALUATING MODEL BEFORE TRAINING
⏳ This will take 3-5 minutes. Patience is a virtue!



  0%|          | 0/70 [00:00<?, ?it/s]

===> corr=7, total=10, Acc=70.00%, Partial=70.00%, Format=90.00%
===> corr=13, total=20, Acc=65.00%, Partial=65.00%, Format=80.00%
===> corr=20, total=30, Acc=66.67%, Partial=66.67%, Format=83.33%
===> corr=25, total=40, Acc=62.50%, Partial=62.50%, Format=85.00%
===> corr=31, total=50, Acc=62.00%, Partial=62.00%, Format=84.00%
===> corr=34, total=60, Acc=56.67%, Partial=56.67%, Format=83.33%
===> corr=37, total=70, Acc=52.86%, Partial=52.86%, Format=80.00%
===> corr=45, total=80, Acc=56.25%, Partial=56.25%, Format=82.50%
===> corr=51, total=90, Acc=56.67%, Partial=56.67%, Format=82.22%
===> corr=58, total=100, Acc=58.00%, Partial=58.00%, Format=84.00%
===> corr=61, total=110, Acc=55.45%, Partial=55.45%, Format=83.64%
===> corr=69, total=120, Acc=57.50%, Partial=57.50%, Format=85.00%
===> corr=73, total=130, Acc=56.15%, Partial=56.15%, Format=84.62%
===> corr=76, total=140, Acc=54.29%, Partial=54.29%, Format=85.71%

📈 PRE-TRAINING RESULTS:
   Correct answers: 76/140
   Accuracy: 54.29%


### Checkpointing and Logging Configuration

Setting up model saving and TensorBoard metrics tracking.

In [21]:
# Checkpoint saving configuration
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=SAVE_INTERVAL_STEPS, max_to_keep=MAX_TO_KEEP
)

# Metrics logger for TensorBoard
metrics_logging_options = metrics_logger.MetricsLoggerOptions(
    log_dir="/tmp/content/tmp/tensorboard/grpo", flush_every_n_steps=20
)

print("✅ Checkpointing configured")
print("✅ Metrics logging configured")

✅ Checkpointing configured
✅ Metrics logging configured


## 📈 Optimizer Configuration

Using **AdamW** with warmup + cosine decay schedule.

**Schedule Design**:
- Warm-up (steps 0-170): Learning rate gradually increases from 0 → 3e-6
- Training (steps 170-1700): Cosine decay from 3e-6 → 0
- Gradient clipping at 0.1 prevents instability

This conservative approach is critical for RL, where unstable updates can cause catastrophic collapse.

In [22]:
# Optimizer with learning rate schedule and gradient clipping
optimizer = optax.adamw(
    learning_rate=optax.schedules.warmup_cosine_decay_schedule(
        init_value=0.0,
        peak_value=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        decay_steps=MAX_STEPS,
        end_value=0.0,
    ),
    b1=B1,
    b2=B2,
    weight_decay=WEIGHT_DECAY,
)

if MAX_GRAD_NORM is not None:
    optimizer = optax.chain(
        optax.clip_by_global_norm(max_norm=MAX_GRAD_NORM),
        optimizer,
    )

print("✅ Optimizer configured with:")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Warmup steps: {WARMUP_STEPS}")
print(f"   Max gradient norm: {MAX_GRAD_NORM}")

✅ Optimizer configured with:
   Learning rate: 3e-06
   Warmup steps: 200.0
   Max gradient norm: 0.1


## 🎮 RL Cluster Architecture

Configuring the distributed reinforcement learning infrastructure.

**Three-Role Architecture**:
1. **Actor**: Current policy model generating responses
2. **Reference**: Frozen base model for KL divergence
3. **Rollout**: Manages generation process and batching

**Rollout Configuration**:
- Temperature=0.9: Balanced exploration vs exploitation
- Top-K=50, Top-P=1.0: Moderate sampling diversity
- EOS tokens: Proper sequence termination

This separation enables efficient parallel processing across 8 TPU cores.

In [23]:
# RL Cluster configuration
cluster_config = rl_cluster_lib.ClusterConfig(
    role_to_mesh={
        rl_cluster_lib.Role.ACTOR: mesh,
        rl_cluster_lib.Role.REFERENCE: mesh,
        rl_cluster_lib.Role.ROLLOUT: mesh,
    },
    rollout_engine='vanilla',
    offload_to_cpu=False,
    training_config=rl_cluster_lib.RLTrainingConfig(
        actor_optimizer=optimizer,
        eval_every_n_steps=EVAL_EVERY_N_STEPS,
        max_steps=MAX_STEPS,
        mini_batch_size=TRAIN_MICRO_BATCH_SIZE,
        train_micro_batch_size=TRAIN_MICRO_BATCH_SIZE,
        metrics_logging_options=metrics_logging_options,
        checkpoint_root_directory=CKPT_DIR,
        checkpointing_options=checkpointing_options,
    ),
    rollout_config=base_rollout.RolloutConfig(
        max_tokens_to_generate=TOTAL_GENERATION_STEPS,
        max_prompt_length=MAX_PROMPT_LENGTH,
        kv_cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        top_k=TOP_K,
        eos_tokens=[1, 106],
    ),
)

# GRPO configuration
grpo_config = GRPOConfig(
    num_generations=NUM_GENERATIONS,
    num_iterations=NUM_ITERATIONS,
    beta=BETA,
    epsilon=EPSILON,
)

print("✅ RL Cluster configured")
print("✅ GRPO configured")
print(f"   Num generations per prompt: {NUM_GENERATIONS}")
print(f"   Beta (KL penalty): {BETA}")
print(f"   Epsilon (clipping): {EPSILON}")

✅ RL Cluster configured
✅ GRPO configured
   Num generations per prompt: 4
   Beta (KL penalty): 0.04
   Epsilon (clipping): 0.2


## 🚀 GRPO Training Loop

Starting the main training process.

**GRPO Algorithm Flow**:
1. **Sample** batch of 2 questions from training data
2. **Generate** 4 responses per question (exploration)
3. **Score** each response using 7 reward functions
4. **Compute advantages**: Normalize rewards to identify best/worst responses
5. **Update policy**: Increase probability of high-reward responses
6. **Apply KL penalty**: Prevent over-optimization away from base model
7. **Repeat** for 1700 steps

**Expected Runtime**: 15-25 minutes on TPU v5e-8

The model learns by comparing responses to the same question, discovering what makes some better than others.

In [24]:
import os
import torch
from pathlib import Path

print("\n" + "="*60)
print("🚀 STARTING GRPO TRAINING")
print("="*60)
print()

# RL cluster setup
rl_cluster = rl_cluster_lib.RLCluster(
    actor=lora_policy,
    reference=ref_model,
    tokenizer=tokenizer,
    cluster_config=cluster_config,
)

print("✅ RL Cluster initialized")

# GRPO Trainer with reward functions for SVAMP
grpo_trainer = GRPOLearner(
    rl_cluster=rl_cluster,
    reward_fns=[
        match_format_exactly,        # Structural constraint
        match_format_approximately,  # Soft structure check
        check_answer_svamp,          # Hard correctness for numbers
        check_number_svamp,          # Flexible number extraction
        soft_reasoning_steps,        # Logical connectors
        meaningful_reasoning_length, # Appropriate length
        reward_algebraic_notation,   # Algebraic equations
    ],
    grpo_config=grpo_config,
)

print("✅ GRPO Trainer initialized")
print(f"   Active reward functions: 7")
print()

print("="*60)
print("⏳ TRAINING IN PROGRESS...")
print("="*60)
print(f"📊 Total steps: {MAX_STEPS}")
print()
print("☕ This will take 1-2 hours. Go grab coffee!")
print("   You can monitor progress in the output below.")
print("="*60)
print()

# TRAINING LOOP
try:
    with mesh:
        grpo_trainer.train(train_dataset)
    
    print("\n" + "="*60)
    print("✅ TRAINING COMPLETED SUCCESSFULLY!")
    print("="*60)
    
except Exception as e:
    print("\n" + "="*60)
    print("❌ TRAINING FAILED!")
    print(f"Error: {e}")
    print("="*60)
    raise


🚀 STARTING GRPO TRAINING



wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: Currently logged in as: saisurya24 (saisurya24-technical-university-of-applied-sciences-w-rz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ RL Cluster initialized
✅ GRPO Trainer initialized
   Active reward functions: 7

⏳ TRAINING IN PROGRESS...
📊 Total steps: 2000

☕ This will take 1-2 hours. Go grab coffee!
   You can monitor progress in the output below.

START ============================
Question: For the walls of the house he would use 8 large planks of wood. If each plank of wood needs 74 piece...
Correct Answer: 8
Response: Yes, I’m ready to put on my mathematical reasoning expert hat! Let’s tackle this problem.

<reasoning>
Step 1: John needs 8 large planks of wood.
Step 2: Each plank needs 74 pieces of nails.
Step 3: T...
Extracted: 592
END ==============================


Actor Training:   0%|          | 0/2000 [00:00<?, ?step/s]

START ============================
Question: Every day Ryan spends 7 hours on learning english, 2 hours on learning chinese and 4 hours on learni...
Correct Answer: 3
Response: Yes, I’m ready to analyze the problem and provide a detailed solution following your specified format.

<reasoning>
Step 1: Ryan spends 7 hours on learning English each day.
Step 2: He spends 2 hours ...
Extracted: 7
END ==============================
START ============================
Question: Jack received 10 emails in the morning, 5 emails in the afternoon and 4 emails in the evening. How m...
Correct Answer: 1
Response: Yes, let’s solve this problem!

<reasoning>
Step 1: Jack received 10 emails in the morning, 5 emails in the afternoon and 4 emails in the evening.
Step 2: Calculate the number of emails received in th...
Extracted: 1
END ==============================


...

START ============================
Question: Winter is almost here and most animals are migrating to warmer countries. There were 87 bird familie...
Correct Answer: 73
Response: Yes, let's tackle this problem!

<reasoning>
Step 1: We are given the number of bird families living near the mountain, which is 87.
Step 2: We are given the number of bird families that flew away for...
Extracted: 80
END ==============================
START ============================
Question: Mary is baking a cake. The recipe calls for 14 cups of flour and 6 cups of sugar. She already put in...
Correct Answer: 3
Response: Yes, let's tackle this problem!

<reasoning>
Step 1: We are given that the recipe calls for 14 cups of flour and 6 cups of sugar.
Step 2: Mary has already put in 5 cups of flour.
Step 3: We need to fi...
Extracted: 9
END ==============================
START ============================
Question: They decided to hold the party in their backyard. They have 4 sets of tables and each set has 3

wandb: WARNING Tried to log to step 0 that is less than the current step 529. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


START ============================
Question: Edward spent $ 6 to buy books and $ 16 to buy pens. Now he has $ 19. How much did Edward have before...
Correct Answer: 41
Response: Yes, let's tackle this problem!

<reasoning>
Step 1: We are given that Edward spent $6 on books and $16 on pens. So the total amount spent is $6 + $16 = $22.
Step 2: After buying the books and pens, E...
Extracted: 41
END ==============================
START ============================
Question: In Haley's class 4 are boys who love to play marbles. If Haley has 23 marbles and wants to keep 15 m...
Correct Answer: 2
Response: Yes, let's tackle this problem!

<reasoning>
Step 1: We are given that there are 4 boys in Haley's class.
Step 2: Haley has a total of 23 marbles.
Step 3: Haley wants to keep 15 marbles for herself.
S...
Extracted: 2
END ==============================
START ============================
Question: Matthew had 15 crackers. If Matthew gave equal numbers of crackers to his 5 friends and still h

actor/train/kl,▁▁▁▁▁▁▂▂▂▂▂▂▃▂▂▃▂▃▂▂▃▂▃▅▄▅▄▅▄▄▆▆▃▄▅▇█▇▇▅
actor/train/loss,▃▃▁▃▄▃▃▄▅▃▇▆▃▄▃▃▇▄▆▆▅▁▃▆█▇▄▄▅▃▄▃▃▆▆▄▆▃▃▆
actor/train/perplexity,▅▄▆▂▁▂▃▃▇▇▁▄█▆█▃▄▇▄▃▆█▃▇▃▅▄▅▄▃▅▂▄▄▄▂▄▃▆▄
actor/train/step_time_sec,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁
actor/train/steps_per_sec,▁███▇█████▇█████████████████▇███████████
actor/train/tflops_per_step,▁
jax/checkpoint/write/blocking_gbytes_per_sec,▁
jax/checkpoint/write/gbytes,▁
jax/checkpoint/write/gbytes_per_sec,▁
jax/core/compile/backend_compile_duration,▁
+16,...



✅ TRAINING COMPLETED SUCCESSFULLY!


## 📈 Post-Training Evaluation

Testing the fine-tuned model on the held-out test set to measure improvement.

**Expected Improvements**:
- Accuracy: +10-20% absolute gain
- Format compliance: +10-15% (near-perfect structure)
- Reasoning quality: More logical, step-by-step solutions

In [25]:
import os
import jax._src.monitoring as monitoring

# Disable wandb
os.environ['WANDB_MODE'] = 'disabled'

# Clear JAX monitoring callbacks - access the internal list directly
try:
    monitoring._scalar_listeners.clear()
    print("✅ Cleared JAX monitoring callbacks")
except Exception as e:
    print(f"⚠️ Could not clear callbacks: {e}")
    # Fallback: replace the list entirely
    monitoring._scalar_listeners = []
    print("✅ Replaced monitoring callbacks with empty list")

print("\n" + "="*60)
print("📊 EVALUATING TRAINED MODEL")
print("="*60)
print("⏳ This will take 3-5 minutes...")
print()

# Recreate sampler with trained model
trained_sampler = sampler_lib.Sampler(
    transformer=lora_policy,
    tokenizer=tokenizer,
    cache_config=sampler_lib.CacheConfig(
        cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        num_layers=model_config.num_layers,
        num_kv_heads=model_config.num_kv_heads,
        head_dim=model_config.head_dim,
    ),
)

(corr_after, total_after, accuracy_after, partial_accuracy_after, format_accuracy_after) = evaluate(
    test_dataset,
    trained_sampler,
    **GENERATION_CONFIGS["greedy"],
)

print("\n" + "="*60)
print("📈 POST-TRAINING RESULTS:")
print(f"   Correct answers: {corr_after}/{total_after}")
print(f"   Accuracy: {accuracy_after:.2f}%")
print(f"   Partial accuracy: {partial_accuracy_after:.2f}%")
print(f"   Format compliance: {format_accuracy_after:.2f}%")
print("="*60)

print("\n" + "="*60)
print("📊 IMPROVEMENT COMPARISON:")
print("="*60)
print(f"   Accuracy:        {accuracy:.2f}% → {accuracy_after:.2f}% (+{accuracy_after - accuracy:.2f}%)")
print(f"   Partial:         {partial_accuracy:.2f}% → {partial_accuracy_after:.2f}% (+{partial_accuracy_after - partial_accuracy:.2f}%)")
print(f"   Format:          {format_accuracy:.2f}% → {format_accuracy_after:.2f}% (+{format_accuracy_after - format_accuracy:.2f}%)")
print("="*60)

✅ Cleared JAX monitoring callbacks

📊 EVALUATING TRAINED MODEL
⏳ This will take 3-5 minutes...



  0%|          | 0/70 [00:00<?, ?it/s]

===> corr=7, total=10, Acc=70.00%, Partial=70.00%, Format=100.00%
===> corr=14, total=20, Acc=70.00%, Partial=70.00%, Format=100.00%
===> corr=21, total=30, Acc=70.00%, Partial=70.00%, Format=96.67%
===> corr=29, total=40, Acc=72.50%, Partial=72.50%, Format=97.50%
===> corr=37, total=50, Acc=74.00%, Partial=74.00%, Format=98.00%
===> corr=42, total=60, Acc=70.00%, Partial=70.00%, Format=98.33%
===> corr=50, total=70, Acc=71.43%, Partial=71.43%, Format=98.57%
===> corr=56, total=80, Acc=70.00%, Partial=70.00%, Format=98.75%
===> corr=64, total=90, Acc=71.11%, Partial=71.11%, Format=97.78%
===> corr=71, total=100, Acc=71.00%, Partial=71.00%, Format=98.00%
===> corr=78, total=110, Acc=70.91%, Partial=70.91%, Format=98.18%
===> corr=85, total=120, Acc=70.83%, Partial=70.83%, Format=98.33%
===> corr=89, total=130, Acc=68.46%, Partial=68.46%, Format=98.46%
===> corr=94, total=140, Acc=67.14%, Partial=67.14%, Format=98.57%

📈 POST-TRAINING RESULTS:
   Correct answers: 94/140
   Accuracy: 67.1

## 🧪 Interactive Testing

Generating responses for sample questions to qualitatively assess the model's learned reasoning patterns.

We test with:
1. Custom DVDs discount problem
2. Original SVAMP examples
3. Various problem types (addition, division, multi-step)

This helps verify the model produces human-readable, mathematically sound reasoning.

In [26]:
print("\n" + "="*60)
print("🧪 TESTING ON SAMPLE QUESTIONS")
print("="*60)

# Sample SVAMP question
sample_question = """Each pack of DVDs costs 76 dollars. If there is a discount of 25 dollars on each pack, how much do you have to pay to buy each pack?"""

print(f"\n📝 Question: {sample_question}")
print("\n⏳ Generating answer...\n")

# Generate with trained model
response = generate(
    sample_question,
    trained_sampler,
    **GENERATION_CONFIGS["standard"]
)

print("="*60)
print("🤖 MODEL RESPONSE:")
print("="*60)
print(response)
print("="*60)

# Try a few more examples
test_questions = [
    "Rachel had to complete 5 pages of math homework. If she had to complete 4 more pages of reading homework than math homework, how many pages did she have to complete in all?",
    "There were 8 friends playing a video game online when 3 players quit. If each player left had 5 lives, how many lives did they have total?",
    "A farmer has 56 apples. He wants to put them in boxes of 8. How many boxes does he need?",
]

print("\n" + "="*60)
print("🧪 ADDITIONAL TEST QUESTIONS:")
print("="*60)

for i, q in enumerate(test_questions, 1):
    print(f"\n--- Test {i} ---")
    print(f"Q: {q}")
    
    resp = generate(q, trained_sampler, **GENERATION_CONFIGS["greedy"])
    print(f"\nModel response:\n{resp}\n")
    print("-" * 60)


🧪 TESTING ON SAMPLE QUESTIONS

📝 Question: Each pack of DVDs costs 76 dollars. If there is a discount of 25 dollars on each pack, how much do you have to pay to buy each pack?

⏳ Generating answer...

🤖 MODEL RESPONSE:
Yes, let's tackle this problem!

<reasoning>
Step 1: The original price of each pack of DVDs is 76 dollars.
Step 2: There is a discount of 25 dollars on each pack. This means the price after the discount is 76 - 25 = 51 dollars.
Step 3: Therefore, you have to pay 51 dollars to buy each pack of DVDs. </reasoning>
<answer>51</answer>

🧪 ADDITIONAL TEST QUESTIONS:

--- Test 1 ---
Q: Rachel had to complete 5 pages of math homework. If she had to complete 4 more pages of reading homework than math homework, how many pages did she have to complete in all?

Model response:
Yes





<
Stepblue
Step

------------------------------------------------------------

--- Test 2 ---
Q: There were 8 friends playing a video game online when 3 players quit. If each player left had 5 lives

## 💾 Saving Trained Model

Persisting the fine-tuned LoRA adapters, tokenizer, and training configuration.

**Saved Artifacts**:
- `lora_final/` or `lora_state.pkl`: Trained adapter weights (~2GB)
- `tokenizer/`: Vocabulary and special tokens
- `training_config.pkl`: Hyperparameters for reproducibility

**Usage**: These adapters can be merged with the base Gemma 3 1B model for deployment.

In [27]:
import os
from pathlib import Path

os.environ['WANDB_MODE'] = 'disabled'  # Disable wandb to avoid the error

print("\n💾 Saving final trained model...")

# Create save directory
save_dir = Path("./trained_models")
save_dir.mkdir(parents=True, exist_ok=True)

try:
    # Save the LoRA policy using orbax
    final_checkpointer = ocp.StandardCheckpointer()
    _, final_state = nnx.split(lora_policy)
    
    # Disable any monitoring callbacks that might trigger wandb
    import jax
    jax.monitoring._scalar_listeners.clear()
    
    final_checkpointer.save(str(save_dir / "lora_final"), final_state)
    final_checkpointer.wait_until_finished()
    
    print(f"✅ Final model saved to {save_dir / 'lora_final'}")
    
    # Save tokenizer
    tokenizer.save_pretrained(str(save_dir / "tokenizer"))
    print(f"✅ Tokenizer saved to {save_dir / 'tokenizer'}")
    
    # Save training config
    import pickle
    config_save_path = save_dir / "training_config.pkl"
    with open(config_save_path, 'wb') as f:
        pickle.dump({
            'grpo_config': grpo_config,
            'cluster_config': cluster_config,
            'hyperparameters': {
                'learning_rate': LEARNING_RATE,
                'num_epochs': NUM_EPOCHS,
                'batch_size': TRAIN_MICRO_BATCH_SIZE,
                'num_generations': NUM_GENERATIONS,
                'beta': BETA,
                'epsilon': EPSILON,
            }
        }, f)
    print(f"✅ Training config saved to {config_save_path}")
    
    print("\n" + "="*60)
    print("🎉 ALL FILES SAVED SUCCESSFULLY!")
    print("="*60)
    print(f"📁 Location: {save_dir}")
    print("="*60)
    
except Exception as e:
    print(f"❌ Error saving model: {e}")
    print("Trying alternative save method...")
    
    # Alternative: Save just the state dict
    import cloudpickle
    with open(save_dir / "lora_state.pkl", 'wb') as f:
        cloudpickle.dump(final_state, f)
    print(f"✅ Model state saved to {save_dir / 'lora_state.pkl'}")


💾 Saving final trained model...
❌ Error saving model: module 'jax.monitoring' has no attribute '_scalar_listeners'
Trying alternative save method...
✅ Model state saved to trained_models/lora_state.pkl
